In [1]:
def preprocess_for_sklearn(data):
    data.columns = map(str.lower, data.columns)
    data.sales = data.sales.str.lower()
    # create dummies for categorical variables
    data = data.join(pd.get_dummies(data["sales"], prefix="dept"))
    data = data.join(pd.get_dummies(data["salary"], prefix="salary"))
    # drop variables that should not be in the X matrix
    # these include: employer ID, categorical variables that have
    # been converted into dummies and one dummy per each
    # categorical variable (to avoid perfect multicollinearity)
    data = data.drop(["emp_id", "salary_high", "dept_accounting",
                      "sales", "salary"], axis=1)
    # the test data has an extra column compared to the train data
    # it's the name of the employee and it has to be dropped
    try:
        data = data.drop(["name"], axis=1)
    except ValueError:
        pass
    return data

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('../Data/train.csv')
print(data.shape)
test = pd.read_csv('../Data/test.csv')
print(test.shape)

(13999, 11)
(500, 11)


In [3]:
X = preprocess_for_sklearn(data.drop(["left"], axis=1))
y = data.left
print(X.shape)
Xpred = preprocess_for_sklearn(test.drop(["left"], axis=1))
ypred = test.left
print(Xpred.shape)

(13999, 18)
(500, 18)


## Random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=15, random_state=45, n_estimators=200)
clf.fit(X, y)

predicted_labels = clf.predict(Xpred)

print("Accuracy: " + str(accuracy_score(ypred, predicted_labels)))
print("Recall: " + str(recall_score(ypred, predicted_labels)))

confusion_matrix(ypred, predicted_labels)

Accuracy: 0.988
Recall: 0.9512195121951219


array([[377,   0],
       [  6, 117]])

In [9]:
print(list(X))
clf.feature_importances_

['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'work_accident', 'promotion_last_5years', 'dept_hr', 'dept_it', 'dept_management', 'dept_marketing', 'dept_product_mng', 'dept_randd', 'dept_sales', 'dept_support', 'dept_technical', 'salary_low', 'salary_medium']


array([0.31486599, 0.1165723 , 0.18519465, 0.15299445, 0.19243088,
       0.00913026, 0.00162846, 0.00120084, 0.00145694, 0.00146177,
       0.00092767, 0.00087791, 0.00151864, 0.00278334, 0.00241922,
       0.00306884, 0.00810917, 0.00335868])

In [10]:
pred_proba = pd.DataFrame(clf.predict_proba(Xpred))
#list(clf.predict_proba(Xpred)[:,1])
pred_proba.head()

,0,1
0,0.991899,0.008101
1,0.976543,0.023457
2,0.000980,0.999020
3,0.978955,0.021045
4,0.993986,0.006014


In [21]:
n = 285
print(predicted_labels[n])
pred_proba.loc[n,1]

1


0.9987496361344554

In [23]:
test_obs = np.array([Xpred.loc[285,:]])
clf.predict_proba(test_obs)[0][1]

0.9987496361344554

In [32]:
#x = np.array([[0.11, 0.8, 6.0, 285.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])

#change hours, first of second row(promotion) and last two (salary)
x = np.array([[0.21, 0.8, 6.0, 250.0, 4.0, 0.0,
 1.0, 0.0, 0.0,
 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])

In [33]:
clf.predict_proba(x)[0][1]

0.035365913818268854

# Classification tree

In [ ]:
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

clf = tree.DecisionTreeClassifier(max_depth = 3)
clf = clf.fit(X, y)

predicted_labels = clf.predict(Xpred)

print("Accuracy: " + str(accuracy_score(ypred, predicted_labels)))
print("Recall: " + str(recall_score(ypred, predicted_labels)))

confusion_matrix(ypred, predicted_labels)

In [ ]:
print(list(X))
clf.feature_importances_

In [ ]:
pred_proba = pd.DataFrame(clf.predict_proba(Xpred))

In [ ]:
n = 329
print(predicted_labels[n])
pred_proba.loc[n,1]

In [ ]:
test_obs = np.array([Xpred.loc[329,:]])
clf.predict_proba(test_obs)[0][1]

In [ ]:
x = np.array([[0.12, 0.6, 4.0, 400.0, 5.0, 0.0,
 0.0, 0.0, 0.0,
 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]])

In [ ]:
clf.predict_proba(x)

In [ ]:
#parameters = {'max_depth': list(range(3,21))}
#clf = grid_search.GridSearchCV(tree.DecisionTreeClassifier(), parameters, n_jobs=2, cv = 10)
#clf.fit(X, y)
#tree_model = clf.best_estimator_
#print(clf.best_score_, clf.best_params_) 

# Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

logreg = LogisticRegression()
logreg.fit(X, y)

predicted_labels = logreg.predict(Xpred)

print("Accuracy: " + str(accuracy_score(ypred, predicted_labels)))
print("Recall: " + str(recall_score(ypred, predicted_labels)))

confusion_matrix(ypred, predicted_labels)

## Playground and multiple piles of crap

In [ ]:
def read_data(table_name):
    engine = create_engine(config.database_config)
    sql = "select * from " + table_name
    data = pd.read_sql_query(sql, con = engine)
    try:
        y = data.left
        data = data.drop(["left"], axis = 1)
        return (data, y)
    except:
        return data

def preprocess_for_sklearn(data):
    data.columns = map(str.lower, data.columns)
    data.sales = data.sales.str.lower()
    #create dummies for categorical variables
    data = data.join(pd.get_dummies(data["sales"], prefix="dept"))
    data = data.join(pd.get_dummies(data["salary"], prefix="salary"))
    #drop variables that should not be in the X matrix
    #these include: left (ie the target variable), employer ID, categorical vars and
    #one dummy per category (to avoid perfect multicollinearity)
    data = data.drop(["emp_id", "salary_high", "dept_accounting", "sales", "salary"], axis = 1)
    try:
        data = data.drop(["name"], axis = 1)
    except:
        pass
    return data

In [ ]:
import config
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.linear_model import LogisticRegression
import math
import pickle
pd.options.display.max_columns = 25

##### Next block does prediction using scikitlearn, should match results below

In [ ]:
X, y = read_data('employees_hist_data')
X = preprocess_for_sklearn(X)
logreg = LogisticRegression()
logreg.fit(X, y)
y_pred = logreg.predict(X.loc[0:1,:])
y_pred_prob = logreg.predict_proba(X.loc[0:10,:])[:, 1]
print(y_pred)
print(y_pred_prob)

In [ ]:
a = list(X.loc[2,:])
test = np.array([a])
y_pred = logreg.predict_proba(test)[:, 1][0]
y_pred

## Check everything worked nicely with pickle approach

##### Predict using scikitlearn on a single observation

In [ ]:
pkl_filename = '../models/logistic.pkl'
model_pkl = open(pkl_filename, 'rb')
model = pickle.load(model_pkl)
model_pkl.close()

In [ ]:
a = list(X.loc[2,:])
a[6] = 1
a[16] = 0
a[17] = 0
test = np.array([a])
y_pred = model.predict_proba(test)[:, 1][0]
y_pred
#60% when low
#46.8% when medium
#20.8% when high

In [ ]:
X.loc[2,:]

Test the app with this data:
- [0.57, 0.82, 4, 269, 2, False, False, Technical, Medium] ==> 21.30%
- [0.97, 0.61, 4, 262, 3, True, True, Marketing, Medium] ==> 0.52%

In [ ]:
a[6] = 0
a